In [1]:
# Import required libraries for Reddit API data collection
import praw  # For making HTTP requests to Reddit API
import pandas as pd  # For data manipulation and analysis
import re
from datetime import datetime
import emoji
import string
import os
from pysentimiento import create_analyzer
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="michellejieli/emotion_text_classifier")
sentiment_analyzer = create_analyzer(task="sentiment", lang="en")


/opt/anaconda3/envs/uoft_senti_db/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def clean_text(text):
    """
    Remove all special characters from the text except for letters, numbers, and spaces.
    """
    return re.sub(r'[^\w\s]', '', text)


def convert_emoji_to_text(text, emoji_wrapper="emoji"):
    """
    Converts emoji in the text to descriptive text.
    
    This function uses emoji.demojize() to replace any emoji with their
    corresponding text (e.g., "😄" becomes ":smile:"). It then wraps the
    emoji description using the provided emoji_wrapper.
    
    Args:
        text (str): The original text containing emoji.
        emoji_wrapper (str): A string that will be used to wrap the emoji text.
        
    Returns:
        str: The text with emojis converted to wrapped text.
    """
    # Convert emojis to descriptive text (e.g., :smile:)
    demojized = emoji.demojize(text)
    # Define a wrapper string (e.g., " emoji ")
    wrapper = f" {emoji_wrapper} ".replace("  ", " ")
    # Replace the demojized emoji pattern :emoji_name: with the wrapped emoji_name
    # For example, ":smile:" becomes " emoji smile emoji "
    result = re.sub(r':([^:\s]+):', lambda m: wrapper + m.group(1) + wrapper, demojized)
    return result

def is_valid_word(word):
    """
    Returns True if the word is either:
    - Composed solely of punctuation
    - Contains at least one English letter (a-z or A-Z)
    """
    # Keep if word is only punctuation
    if all(ch in string.punctuation for ch in word):
        return True
    # Keep if the word contains at least one ASCII letter
    if re.search(r'[a-zA-Z]', word):
        return True
    return False

def filter_non_english(text):
    """
    Splits the text into words and filters out any word that is not an English word,
    an emoji, or punctuation.
    
    Args:
        text (str): The input text.
    
    Returns:
        str: The cleaned text.
    """
    words = text.split()
    filtered_words = [word for word in words if is_valid_word(word)]
    return ' '.join(filtered_words)

# 4. Convert created_utc to readable date format (YYYY-MM-DD)
def convert_utc_to_readable(utc_timestamp):
    """
    Convert UTC timestamp to a readable date format (YYYY-MM-DD).
    
    Args:
        utc_timestamp (float): UTC timestamp in seconds since epoch.
        
    Returns:
        str: Formatted date string or empty string if input is NaN.
    """
    
    try:
        # Handle NaN values
        if pd.isna(utc_timestamp):
            return ""
        
        # Convert to datetime object
        dt = datetime.fromtimestamp(utc_timestamp)
        
        # Format as YYYY-MM-DD HH:MM:SS
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    
    except (ValueError, TypeError, OSError):
        return ""
    

In [3]:
def sentiment_analysis(master_df, text_column='body'):
    """
    Iterates through each text in the DataFrame, applies sentiment_analyzer,
    and adds two columns: sentiment_prediction and sentiment_prob.
    
    Args:
        master_df (pd.DataFrame): The DataFrame to analyze
        text_column (str): The column containing text to analyze
        
    Returns:
        pd.DataFrame: DataFrame with added sentiment columns
    """
    senti_pred = []
    senti_prob = []
    
    for index, row in master_df.iterrows():
        text = row[text_column]
        
        if pd.isna(text) or text == "":
            senti_pred.append("NEU")
            senti_prob.append({"NEU": 0.34, "NEG": 0.33, "POS": 0.33})
        else:
            result = sentiment_analyzer.predict(text)
            senti_pred.append(result.output)
            senti_prob.append(result.probas)
    
    master_df['sentiment_prediction'] = senti_pred
    master_df['sentiment_prob'] = senti_prob
    
    return master_df

def emotion_analysis(input_df, text_column='body'):
    emotion_pred = []
    emotion_prob = []
    
    for index, row in input_df.iterrows():
        text = row[text_column]
        result = classifier(text)
        emotion_pred.append(result[0]['label'])
        emotion_prob.append({label['label']: label['score'] for label in result})

    input_df['emotion_prediction'] = emotion_pred
    input_df['emotion_prob'] = emotion_prob

    return input_df

def preprocess_dataframe(df):
    """
    Apply all preprocessing steps to a DataFrame.
    
    Args:
        df (pd.DataFrame): The DataFrame to preprocess
        
    Returns:
        pd.DataFrame: The preprocessed DataFrame
    """
    # Make a copy to avoid modifying the original
    processed_df = df.copy()
    
    # 1. Keep only posts with URLs starting with UofT comments
    processed_df = processed_df[processed_df['url'].str.startswith('https://www.reddit.com/r/UofT/comments/')].reset_index(drop=True).copy()
    
    # 2. Clean body text: lowercase, strip, remove special chars except spaces
    processed_df['body'] = processed_df['body'].str.lower()
    processed_df['body'] = processed_df['body'].str.strip()
    
    # 3. Remove special characters except spaces
    processed_df['body'] = processed_df['body'].apply(clean_text)
    
    # 4. Converts emoji in the text to descriptive text
    processed_df['body'] = processed_df['body'].apply(convert_emoji_to_text)
    
    # 5. Filter out non-English words
    processed_df['body'] = processed_df['body'].apply(filter_non_english)
    
    # 6. Convert created_utc to readable date format
    processed_df['created_utc'] = processed_df['created_utc'].apply(convert_utc_to_readable)
    
    # 7. Preprocess the body column using pysentimiento (optional - currently commented out)
    # processed_df['body'] = processed_df['body'].apply(preprocess_tweet)
    
    return processed_df


def process_all_monthly_submissions(folder_path='individual_submissions_month'):
    """
    Process all CSV files in the monthly submissions folder.
    Apply preprocessing and sentiment analysis to each file.
    
    Args:
        folder_path (str): Path to the folder containing CSV files
        
    Returns:
        pd.DataFrame: Combined DataFrame with all processed submissions
    """
    # Get list of all CSV files in the folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    csv_files.sort()  # Sort to process in order
    
    print(f"Found {len(csv_files)} CSV files to process")
    
    all_processed_data = []
    
    for i, filename in enumerate(csv_files):
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file {i+1}/{len(csv_files)}: {filename}")
        
        try:
            # Load the CSV file
            df = pd.read_csv(file_path)
            print(f"  - Loaded {len(df)} rows")
            
            # Apply preprocessing
            processed_df = preprocess_dataframe(df)
            print(f"  - After filtering: {len(processed_df)} rows")
            
            # Apply sentiment analysis
            if len(processed_df) > 0:
                processed_df = sentiment_analysis(processed_df, text_column='body')
                print(f"  - Sentiment analysis completed")

                processed_df = emotion_analysis(processed_df, text_column='body')
                print(f"  - Emotion analysis completed")
                
                # Add source file information
                processed_df['source_file'] = filename
                
                all_processed_data.append(processed_df)
            else:
                print(f"  - No data remaining after filtering")
                
        except Exception as e:
            print(f"  - Error processing {filename}: {str(e)}")
            continue
    
    # Combine all processed data
    if all_processed_data:
        combined_df = pd.concat(all_processed_data, ignore_index=True)
        print(f"\nProcessing complete! Combined dataset has {len(combined_df)} rows")
        return all_processed_data, combined_df
    else:
        print("\nNo data was successfully processed")
        return pd.DataFrame()


# Example usage:
# processed_monthly_data = process_all_monthly_submissions('individual_submissions_month')
# processed_monthly_data.to_excel('processed_monthly_submissions_with_sentiment.xlsx', index=False)

In [4]:
# Process all monthly submissions
print("Starting to process all monthly submission CSV files...")
all_processed_data, processed_monthly_data = process_all_monthly_submissions('individual_submissions_month')

# Display summary statistics
if len(processed_monthly_data) > 0:
    print(f"\n=== PROCESSING SUMMARY ===")
    print(f"Total processed rows: {len(processed_monthly_data)}")
    print(f"Unique submissions: {processed_monthly_data['submission_title'].nunique()}")
    print(f"Date range: {processed_monthly_data['created_utc'].min()} to {processed_monthly_data['created_utc'].max()}")
    
    # Sentiment distribution
    print(f"\n=== SENTIMENT DISTRIBUTION ===")
    sentiment_counts = processed_monthly_data['sentiment_prediction'].value_counts()
    print(sentiment_counts)
    print(f"Sentiment percentages:")
    for sentiment, count in sentiment_counts.items():
        percentage = (count / len(processed_monthly_data)) * 100
        print(f"  {sentiment}: {percentage:.1f}%")
else:
    print("No data was processed successfully.")

Starting to process all monthly submission CSV files...
Found 100 CSV files to process
Processing file 1/100: submission_0.csv
  - Loaded 59 rows
  - After filtering: 58 rows
  - Sentiment analysis completed
  - Emotion analysis completed
Processing file 2/100: submission_1.csv
  - Loaded 15 rows
  - After filtering: 14 rows
  - Sentiment analysis completed
  - Emotion analysis completed
Processing file 3/100: submission_10.csv
  - Loaded 26 rows
  - After filtering: 26 rows
  - Sentiment analysis completed
  - Emotion analysis completed
Processing file 4/100: submission_11.csv
  - Loaded 5 rows
  - After filtering: 4 rows
  - Sentiment analysis completed
  - Emotion analysis completed
Processing file 5/100: submission_12.csv
  - Loaded 27 rows
  - After filtering: 27 rows
  - Sentiment analysis completed
  - Emotion analysis completed
Processing file 6/100: submission_13.csv
  - Loaded 32 rows
  - After filtering: 31 rows
  - Sentiment analysis completed
  - Emotion analysis completed

In [5]:
processed_monthly_data

,submission_title,author,id,url,created_utc,body,type,score,sentiment_prediction,sentiment_prob,emotion_prediction,emotion_prob,source_file
0,Why is UofT using AI?? Could they not have fou...,internetMujahideen,n5k4y1g,https://www.reddit.com/r/UofT/comments/1mb6vic...,2025-07-28 01:14:54,they dont make diverse utopian friend groups s...,comment,316,NEG,"{'NEG': 0.7034717202186584, 'NEU': 0.291085332...",neutral,{'neutral': 0.6760635375976562},submission_0.csv
1,Why is UofT using AI?? Could they not have fou...,random_name_245,n5kzzls,https://www.reddit.com/r/UofT/comments/1mb6vic...,2025-07-28 06:07:07,lol i wonder what the prompt was like have stu...,comment,116,NEU,"{'NEG': 0.16222292184829712, 'NEU': 0.65833210...",neutral,{'neutral': 0.9531378149986267},submission_0.csv
2,Why is UofT using AI?? Could they not have fou...,ihatedougford,n5k06ru,https://www.reddit.com/r/UofT/comments/1mb6vic...,2025-07-28 00:35:56,classic university college the cheapest and la...,comment,115,NEG,"{'NEG': 0.8319604992866516, 'NEU': 0.161235094...",neutral,{'neutral': 0.9534212350845337},submission_0.csv
3,Why is UofT using AI?? Could they not have fou...,BabaYagaTO,n5n0g1d,https://www.reddit.com/r/UofT/comments/1mb6vic...,2025-07-28 13:00:11,and now its gone httpswwwucutorontocaprogramss...,comment,16,NEU,"{'NEG': 0.3041565418243408, 'NEU': 0.678175747...",neutral,{'neutral': 0.9478220343589783},submission_0.csv
4,Why is UofT using AI?? Could they not have fou...,CaptainKoreana,n5kqi34,https://www.reddit.com/r/UofT/comments/1mb6vic...,2025-07-28 04:36:00,yikes uc,comment,14,NEG,"{'NEG': 0.7929663062095642, 'NEU': 0.183752104...",neutral,{'neutral': 0.9800550937652588},submission_0.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,Is this scam? 111111111111111¹1111111111111111...,Potential_Fee4153,n75x28d,https://www.reddit.com/r/UofT/comments/1miqnu1...,2025-08-05 22:46:02,reason i thought it was scam cuz i already did...,comment,3,NEG,"{'NEG': 0.5585876107215881, 'NEU': 0.434544444...",neutral,{'neutral': 0.8371781706809998},submission_99.csv
1588,Is this scam? 111111111111111¹1111111111111111...,DeepGas4538,n75yhwt,https://www.reddit.com/r/UofT/comments/1miqnu1...,2025-08-05 22:54:44,perhaps you misread,comment,11,NEU,"{'NEG': 0.3851972818374634, 'NEU': 0.596996665...",neutral,{'neutral': 0.851717472076416},submission_99.csv
1589,Is this scam? 111111111111111¹1111111111111111...,crud_lover,n761s49,https://www.reddit.com/r/UofT/comments/1miqnu1...,2025-08-05 23:15:36,all students are required to enroll with duo b...,comment,-9,NEU,"{'NEG': 0.09463470429182053, 'NEU': 0.89856177...",neutral,{'neutral': 0.9491429328918457},submission_99.csv
1590,Is this scam? 111111111111111¹1111111111111111...,132ads,n76438p,https://www.reddit.com/r/UofT/comments/1miqnu1...,2025-08-05 23:30:46,double check the first comment you replied to,comment,11,NEU,"{'NEG': 0.012451458722352982, 'NEU': 0.9616963...",neutral,{'neutral': 0.9694627523422241},submission_99.csv


In [ ]:
sentiment_counts = processed_monthly_data['sentiment_prediction'].value_counts()
emotion_counts = processed_monthly_data['emotion_prediction'].value_counts()


In [8]:
processed_monthly_data.to_csv('processed_monthly_submissions_with_sentiment_emotion.csv', index=False)

In [7]:
emotion_counts

emotion_prediction
neutral     1171
joy          159
sadness      111
anger         81
surprise      36
fear          20
disgust       14
Name: count, dtype: int64

In [ ]:
processed_monthly_data